In [1]:
from functools import partial
import os, argparse, torch, gc
import numpy as np
import torch.optim as optim
from torch.nn import MSELoss
from easydict import EasyDict as edict
from torch.utils.tensorboard import SummaryWriter
from leo_segmentation.data import Datagenerator, TrainingStats
from leo_segmentation.model import LEO, load_model, save_model
from leo_segmentation.utils import load_config, check_experiment, get_named_dict
from run import train_model


## Train

In [2]:
config = load_config()

In [3]:
leo, meta_data, classes = train_model(config)

NameError: name 'train_model' is not defined

## Predictions on Meta Val / Test

In [4]:
leo

NameError: name 'leo' is not defined

In [5]:
dataloader = Datagenerator(config, dataset="pascal_voc", data_type="meta_val")
val_meta_data = dataloader.get_batch_data()
class_in_metadata = val_meta_data[-1]

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [6]:
class_in_metadata

NameError: name 'class_in_metadata' is not defined

In [7]:
select_class = -1
select_sample = 0

In [8]:
print("class", class_in_metadata[select_class], "is selected")
batch_data = get_named_dict(val_meta_data, select_class)
input_data = batch_data.val_data[select_sample].permute(1, 2, 0)
plt.title("First three channels of embeddings is visualized")
plt.imshow(input_data.cpu().detach().numpy()[:,:,:3])

NameError: name 'class_in_metadata' is not defined

In [9]:
batch_data.val_data.shape

NameError: name 'batch_data' is not defined

In [10]:
latents, kl_loss = leo.forward_encoder(batch_data.tr_data)
tr_loss, predicted_weights, _ = leo.forward_decoder(batch_data.tr_data, latents, batch_data.tr_data_masks)
val_loss = leo.finetuning_inner_loop(batch_data, tr_loss, predicted_weights)
_, _, predictions = leo.forward_decoder(batch_data.val_data, latents, batch_data.val_data_masks)

NameError: name 'leo' is not defined

In [11]:
select_sample = 0
def plot_batch_mask(batch_data, select_sample):
    input_mask = batch_data.val_data_masks[select_sample]
    plt.title("ground truth mask")
    plt.imshow(input_mask.cpu().detach().numpy(), cmap="gray")
    return

plot_batch_mask(batch_data, select_sample)

NameError: name 'batch_data' is not defined

In [12]:
def plot_prediction_mask(pred_mask, select_sample):
    plt.title("mask prediction")
    plt.imshow(np.argmax(pred_mask[select_sample].cpu().detach().numpy(), 0), cmap="gray")
    return

plot_prediction_mask(predictions, select_sample)

NameError: name 'predictions' is not defined

In [13]:
calc_iou_per_class(predictions, batch_data.val_data_masks)

NameError: name 'calc_iou_per_class' is not defined